In [10]:
import numpy as np
import os
import pandas as pd

import matplotlib
# For draw in bg
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# For draw in bg
plt.switch_backend('agg')

from scipy.signal import medfilt
matplotlib.rcParams.update({'font.size': 8})

import seaborn as sns
sns.set(style="darkgrid")

from visdom import Visdom

INF = 1e9

In [11]:
def plot(viz, win, dirname, targs_names, quant=2, smooth=4, opt='each'):
    
    if opt == 'each': 
        data, y_name = load_data(dirname, targs_names, smooth_max=smooth)        
    elif opt == 'best': 
        data, y_name = load_data_with_transform_best(dirname, targs_names, quant=quant, smooth_max=smooth)

    sns_plot = sns.relplot(x="Epoch", y=y_name, hue='type', kind="line", data=data)
    fig = sns_plot.fig
    plt.title(dirname)
    figname = os.path.join(dirname, 'eval_result.png')
    fig.savefig(figname)
    image = plt.imread(figname)

    # Show it in visdom
    image = np.transpose(image, (2, 0, 1))    
    return viz.image(image, win=win)
def load_data(dirname, targs_names, smooth_max=4):
    df = pd.read_csv(os.path.join(dirname, 'progress.csv'))

    targs = (list(targs_names.keys()))
    df = df[df[targs].notnull()]
    
    smooth_range = 1 if len(df) <= 20 else smooth_max
    
    plot_data = [] 
    for targ, name in sorted(targs_names.items()):
        d = df[[targ]].copy(deep=True)
        d['Epoch'] = range(0, len(d))
        d = d.rolling(smooth_range, center=True, min_periods=1).mean()
        d['Epoch'] = d['Epoch'].astype(int)
        d.insert(2, 'type', name)
        d = d.rename(index=str, columns={targ: 'Reward'})
        plot_data.append(d)    
    df = pd.concat(plot_data)
    return df, 'Reward'

In [21]:
df=pd.read_csv(os.path.join('/Users/zsbjltwjj/Desktop/logs/nosil', 'progress.csv'))

In [25]:
df['MeanRewMF'].dropna(how="all")

4    18.588898
9    48.251260
Name: MeanRewMF, dtype: float64